<a href="https://colab.research.google.com/github/ToastbrotFranzi/table_extraction_from_pdf/blob/main/extract_tables_from_pdf_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabula-py
!pip install JPype1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
import tabula
import pandas as pd
import numpy as np

pd.options.mode.copy_on_write = True

In [ ]:
filepath = "https://www.kba.de/SharedDocs/Downloads/DE/SV/sv222_m1_kraft_pdf.pdf?__blob=publicationFile&v=12"

In [ ]:
# Laden der PDF-Datei
df = tabula.read_pdf(filepath, pages="18-1105")

In [ ]:
len(df)

1909

In [ ]:
count_7_14 = 0
count_73_18 = 0
count_73_19 = 0
count_80_15 = 0
count_else = 0
new_type=[(7, 14), (73, 18), (73, 19), (80, 15)]
all_other_types = []

for d in df:

  if d.shape == (7, 14):
    count_7_14 += 1
  elif d.shape == (73, 18):
    count_73_18 += 1
  elif d.shape == (73, 19):
    count_73_19 += 1
  elif d.shape == (80, 15):
    count_80_15 += 1
  else:
    count_else +=1
    all_other_types.append(d.shape)
    if d.shape not in new_type:
      new_type.append(d.shape)


print("count_7_14: ", count_7_14)
print("count_73_18: ", count_73_18)
print("count_73_19: ", count_73_19)
print("count_80_15: ", count_80_15)
print("count_else: ", count_else)
print("Catched different types of shape: ", new_type)

already_counted = []
for item in all_other_types:
  if item not in already_counted:
    print("")
    print(f"Occurency of {item}:")
    print(all_other_types.count(item))
    already_counted.append(item)

count_7_14:  821
count_73_18:  258
count_73_19:  443
count_80_15:  172
count_else:  215
Catched different types of shape:  [(7, 14), (73, 18), (73, 19), (80, 15), (69, 10), (80, 14), (73, 17), (72, 17), (80, 16), (72, 19), (73, 20), (71, 18), (72, 18), (70, 19), (72, 20), (67, 20), (70, 18), (69, 19), (71, 19), (68, 19), (64, 17), (79, 15), (71, 17), (55, 18)]

Occurency of (69, 10):
1

Occurency of (80, 14):
67

Occurency of (73, 17):
17

Occurency of (72, 17):
3

Occurency of (80, 16):
24

Occurency of (72, 19):
25

Occurency of (73, 20):
33

Occurency of (71, 18):
4

Occurency of (72, 18):
19

Occurency of (70, 19):
3

Occurency of (72, 20):
1

Occurency of (67, 20):
1

Occurency of (70, 18):
3

Occurency of (69, 19):
3

Occurency of (71, 19):
6

Occurency of (68, 19):
1

Occurency of (64, 17):
1

Occurency of (79, 15):
1

Occurency of (71, 17):
1

Occurency of (55, 18):
1


In [ ]:
cols_names = ["HSN", "Fahrzeughersteller", "TSN", "Fahrzeugklasse",
              "Handelsname", "Kraftfahrstoffcode", "Leistung kW", "Emissionscode",
              "CO2 g/km 8.1", "CO2 g/km 8.2", "CO2 g/km 8.3", "CO2 g/km 8.4",
              "Verbrauch l, m3/100km 8.5", "Verbrauch l, m3/100km 8.6",
              "Verbrauch l, m3/100km 8.7", "Verbrauch l, m3/100km 8.8",
              "Elektrischer Energieverbrauch kombiniert Wh/km"]

In [ ]:
def drop_na_cols(d):
  for col in d.columns[:10]:
    if d.shape[0] == d[col].isna().sum():
      d.drop(columns=col, inplace=True)
  for col in d.columns[2:]:
    if d.shape[0] == d[col].isna().sum():
      d.drop(columns=col, inplace=True)
  return d

In [ ]:
# Function to add dummy cols

def add_dummy_cols(df, n):
  for i in range(0,n):
      df[f"dummy_{i}"] = np.nan
  #print(df.columns)
  return df

In [ ]:
dfs_clean = []

def clean_dfs(dfs):
  # iterate through dfs
  for d in df:

    # check if shape is (7,14)
    if d.shape != (7, 14) and d.shape != (7, 17):
      # Check for number of rows
      if d.shape[0] > 73:
        if d.shape[0] == 80:
          d = d.iloc[7:]
        elif d.shape[0] == 79:
          d = d.iloc[6:]
      elif d.shape[0] == 7:
        print(d.shape, "Pre-table wa not removed")

      # Check number of cols
      if d.shape[1] != 10:
        if d.shape[1] == 14:
          d = add_dummy_cols(d, 3)
        elif d.shape[1] == 15:
          d = drop_na_cols(d)
          if d.shape[1] < 17:
            diff = 17 - d.shape[1]
            d = add_dummy_cols(d, diff)
        elif d.shape[1] == 16:
          d = drop_na_cols(d)
          if d.shape[1] < 17:
            diff = 17 - d.shape[1]
            d = add_dummy_cols(d, diff)
        elif d.shape[1] == 18:
           d = drop_na_cols(d)
           if d.shape[1] < 17:
            diff = 17 - d.shape[1]
            d = add_dummy_cols(d, diff)
        elif d.shape[1] == 19:
           d = drop_na_cols(d)
           if d.shape[1] < 17:
            diff = 17 - d.shape[1]
            d = add_dummy_cols(d, diff)
        elif d.shape[1] == 20:
          d = drop_na_cols(d)
          if d.shape[1] < 17:
            diff = 17 - d.shape[1]
            d = add_dummy_cols(d, diff)

        #  rename rows
        if len(d.columns) == 17:
            d.columns = cols_names
        else:
            print(f"number of cols is {len(d.columns)}.")
            print(d.columns)

        if d.shape[1] == 10:
        # add index columns (give them the correct names)
          d = (d.T.reset_index().T.reset_index(drop=True)
            .set_axis(['HSN', 'Fahrzeughersteller', 'TSN', 'Fahrzeugklasse', 'Handelsname',
            'Kraftfahrstoffcode', 'Leistung kW', 'Emissionscode', 'CO2 g/km 8.1', 'Verbrauch l, m3/100km 8.5'],
                      axis=1))

          # add dummy column with correct names
          cols_left = ['CO2 g/km 8.2', 'CO2 g/km 8.3', 'CO2 g/km 8.4','Verbrauch l, m3/100km 8.6',
            'Verbrauch l, m3/100km 8.7', 'Verbrauch l, m3/100km 8.8',
            'Elektrischer Energieverbrauch kombiniert Wh/km']

          for col in cols_left:
            d[col] = np.nan

        # append df to dfs_clean
        dfs_clean.append(d)

In [ ]:
clean_dfs(df)

In [ ]:
len(dfs_clean)

1086

In [ ]:
df_combined = pd.concat(dfs_clean)
df_combined.shape

(79140, 17)

In [ ]:
df_combined.columns

Index(['HSN', 'Fahrzeughersteller', 'TSN', 'Fahrzeugklasse', 'Handelsname',
       'Kraftfahrstoffcode', 'Leistung kW', 'Emissionscode', 'CO2 g/km 8.1',
       'CO2 g/km 8.2', 'CO2 g/km 8.3', 'CO2 g/km 8.4',
       'Verbrauch l, m3/100km 8.5', 'Verbrauch l, m3/100km 8.6',
       'Verbrauch l, m3/100km 8.7', 'Verbrauch l, m3/100km 8.8',
       'Elektrischer Energieverbrauch kombiniert Wh/km'],
      dtype='object')

In [ ]:
df_combined.tail()

,HSN,Fahrzeughersteller,TSN,Fahrzeugklasse,Handelsname,Kraftfahrstoffcode,Leistung kW,Emissionscode,CO2 g/km 8.1,CO2 g/km 8.2,CO2 g/km 8.3,CO2 g/km 8.4,"Verbrauch l, m3/100km 8.5","Verbrauch l, m3/100km 8.6","Verbrauch l, m3/100km 8.7","Verbrauch l, m3/100km 8.8",Elektrischer Energieverbrauch kombiniert Wh/km
50,9891,SMART,AAM,M1,forfour 1.5,1,90,0462,151,6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,9891,SMART,AAM,M1,forfour 1.5,1,90,0462,149,6.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,9891,SMART,AAN,M1,forfour 1.0,1,47,0462,128,5.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,9891,SMART,AAN,M1,forfour 1.0,1,47,0462,133,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,9891,SMART,AAO,M1,forfour BRABUS,1,130,0462,159,6.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_combined.isna().sum()

HSN                                                   0
Fahrzeughersteller                                    0
TSN                                                   0
Fahrzeugklasse                                        0
Handelsname                                          32
Kraftfahrstoffcode                                   72
Leistung kW                                           0
Emissionscode                                        46
CO2 g/km 8.1                                      13528
CO2 g/km 8.2                                      30829
CO2 g/km 8.3                                      68237
CO2 g/km 8.4                                      76545
Verbrauch l, m3/100km 8.5                         72851
Verbrauch l, m3/100km 8.6                         78946
Verbrauch l, m3/100km 8.7                         79100
Verbrauch l, m3/100km 8.8                         79140
Elektrischer Energieverbrauch kombiniert Wh/km    79139
dtype: int64

In [ ]:
df_combined.to_csv(f"/content/df_combined.csv", sep=";")